In [2]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
Pkg.instantiate()
Pkg.build("Cairo")
#---

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
  Building LibCURL ─→ `~/.julia/packages/LibCURL/OoXMv/deps/build.log`
  Building WinRPM ──→ `~/.julia/packages/WinRPM/Y9QdZ/deps/build.log`
  Building Homebrew → `~/.julia/packages/Homebrew/l8kUw/deps/build.log`
  Building Cairo ───→ `~/.julia/packages/Cairo/CXPG1/deps/build.log`


In [30]:
# import necessary packages
using AA228FinalProject
using TabularTDLearning
using POMDPs
using POMDPModels
using POMDPPolicies
using FIB
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf
using QMDP

┌ Info: Precompiling POMDPModels [355abbd5-f08e-5560-ac9e-8b5f2592a0ca]
└ @ Base loading.jl:1192


In [39]:
sensor = Bumper()
config = 3 # 1,2, or 3
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config));#RoombaMDP(config=config);
#pomdp = TigerPOMDP()


In [38]:
sensor = Bumper()
config = 3 # 1,2, or 3
ds = DiscreteRoombaStateSpace(50, 50, 50)
v_steps = range(0.0, stop = 10.0, length = 10) 
om_steps = range(0.0, stop = 1.0, length = 10) 
as = AA228FinalProject.gen_amap([RoombaAct(v,om) for v in v_steps for om in om_steps])

m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(sspace=ds, aspace=[RoombaAct(v,om) for v in v_steps for om in om_steps], config=config))

INFO: POMDPs.jl requirements for solve(::ValueIterationSolver, ::Union{MDP,POMDP}) and dependencies. ([✔] = implemented correctly; [X] = missing)

For solve(::ValueIterationSolver, ::Union{MDP,POMDP}):
  [✔] discount(::RoombaPOMDP)
  [✔] n_states(::RoombaPOMDP)
  [✔] n_actions(::RoombaPOMDP)
  [✔] transition(::RoombaPOMDP, ::RoombaState, ::RoombaAct)
  [✔] reward(::RoombaPOMDP, ::RoombaState, ::RoombaAct, ::RoombaState)
  [✔] stateindex(::RoombaPOMDP, ::RoombaState)
  [✔] actionindex(::RoombaPOMDP, ::RoombaAct)
  [✔] actions(::RoombaPOMDP, ::RoombaState)
For ordered_states(::Union{MDP,POMDP}) (in solve(::ValueIterationSolver, ::Union{MDP,POMDP})):
  [✔] states(::RoombaPOMDP)
For ordered_actions(::Union{MDP,POMDP}) (in solve(::ValueIterationSolver, ::Union{MDP,POMDP})):
  [✔] actions(::RoombaPOMDP)
Note: Missing methods are often due to incorrect importing. You must explicitly import POMDPs functions to add new methods.

Throwing the first exception (from processing solve(::ValueIterati

MethodError: MethodError: no method matching iterate(::AA228FinalProject.RoombaActions)
Closest candidates are:
  iterate(!Matched::Core.SimpleVector) at essentials.jl:589
  iterate(!Matched::Core.SimpleVector, !Matched::Any) at essentials.jl:589
  iterate(!Matched::ExponentialBackOff) at error.jl:171
  ...

MethodError: MethodError: no method matching support(::AA228FinalProject.RoombaInitialDistribution{RoombaPOMDP{Bumper,Bool}})
Closest candidates are:
  support(!Matched::POMDPModelTools.BoolDistribution) at /Users/sarahradz/.julia/packages/POMDPModelTools/eHEjm/src/distributions/bool.jl:25
  support(!Matched::POMDPModels.TMazeStateDistribution) at /Users/sarahradz/.julia/packages/POMDPModels/dUyB8/src/TMazes.jl:55
  support(!Matched::POMDPModels.TMazeInit) at /Users/sarahradz/.julia/packages/POMDPModels/dUyB8/src/TMazes.jl:80
  ...

In [40]:
num_particles = 2000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

In [41]:
# Define the policy to test
mutable struct ToEnd <: Policy
    ts::Int64 # to track the current time-step.
end

# extract goal for heuristic controller
goal_xy = get_goal_xy(m)


# define a new function that takes in the policy struct and current belief,
# and returns the desired action
function POMDPs.action(p::ToEnd, b::ParticleCollection{RoombaState})

    # for 50 steps, act randomly and quickly to learn surrounding info
    if p.ts < 50
        p.ts += 1
        return RoombaAct(5, rand()) 
    end
    p.ts += 1
    
    # after 50 steps of random learning, we follow a proportional controller 
    # to navigate directly to the goal, using the mean belief state
    
    # compute mean belief of a subset of particles
    s = mean(b)
    goal_x, goal_y = goal_xy
    x,y,th = s[1:3]
    ang_to_goal = atan(goal_y - y, goal_x - x)
    del_angle = wrap_to_pi(ang_to_goal - th)
    
    # apply proportional control to compute the turn-rate
    Kprop = 1.0
    om = Kprop * del_angle
    
    # always travel at some fixed velocity
    v = 5.0
    
    return RoombaAct(v, om)
end

In [42]:
# first seed the environment
Random.seed!(5)

# reset the policy
p = ToEnd(0) # here, the argument sets the time-steps elapsed to 0

# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, p, belief_updater, max_steps=100))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.1) # to slow down the simulation
end

#solver = QLearningSolver(m, learning_rate=0.1, n_episodes=5000, max_episode_length=50, eval_every=50, n_eval_traj=100)
#solver = FIBSolver()
solver = QMDPSolver(max_iterations=20, tolerance=1e-3) 
#fib_policy = solve(solver, m)
#rand_policy = RandomPolicy(m);
policy = solve(solver, m) # compute a pomdp policy

#policy = create_policy(solver, m)